In [ ]:
import os
import warnings
warnings.filterwarnings("ignore")
warnings.filterwarnings("ignore", category=DeprecationWarning) 
import numpy as np
import pandas as pd
font = {'size'   : 10}
import matplotlib
matplotlib.rc('font', **font)
import matplotlib.pyplot as plt;
import shutil
import psutil

import sys
import pyemu
import flopy
assert "dependencies" in flopy.__file__
assert "dependencies" in pyemu.__file__
sys.path.insert(0,"..")
import herebedragons as hbd


In [ ]:
# specify the temporary working folder
t_d = os.path.join('pst_template_opt')
if os.path.exists(t_d):
    shutil.rmtree(t_d)

org_t_d = os.path.join("master_ies0")
if not os.path.exists(org_t_d):
    raise Exception()

shutil.copytree(org_t_d,t_d)

In [ ]:
pst = pyemu.Pst(os.path.join(t_d,"pest.pst"))
pe = pst.ies.get("paren",pst.ies.phiactual.iteration.max())

In [ ]:
par = pst.parameter_data
par.loc[pe.columns,"parval1"] = pe.loc["base",:].values.flatten()

In [ ]:
pst.control_data.noptmax = 0
pst.write(os.path.join(t_d,"pest.pst"),version=2)
pyemu.os_utils.run("pestpp-ies pest.pst",cwd=t_d)

In [ ]:
pst.set_res(os.path.join(t_d,"pest.base.rei"))

In [ ]:
forecasts = pst.pestpp_options["forecasts"].split(",")
forecasts.sort()
forecasts

In [ ]:
res = pst.res
res.loc[forecasts,:]

In [ ]:
res.loc[res.name.str.contains("gde"),:]

So our calibrated model is over pumping: the simulated pit level is way lower than 80 and the post-closure GDE flux is not even close to the pre-development estimate...

Let's see if we can optimize our way out of this issue

In [ ]:
wpar = par.loc[par.parnme.str.contains("wel"),:]

In [ ]:
wpar.shape

In [ ]:
par.loc[wpar.parnme,"partrans"] = "none"
pst.pestpp_options["opt_dec_var_groups"] = wpar.pargp.unique().tolist()
pargp = pst.parameter_groups
pargp.loc[pst.pestpp_options["opt_dec_var_groups"],"inctyp"] = "absolute"
pargp.loc[pst.pestpp_options["opt_dec_var_groups"],"derinc"] = 250


In [ ]:
obs = pst.observation_data
obs.loc[forecasts[0],"obgnme"] = "less_than"
obs.loc[forecasts[0],"obsval"] = obs.loc[forecasts[0].replace("time:40151","time:1"),"obsval"]
obs.loc[forecasts[0],"weight"] = 1.0

In [ ]:
obs.loc[forecasts[1],"obsval"] = 80
obs.loc[forecasts[1],"obgnme"] = "less_than"
obs.loc[forecasts[1],"weight"] = 1.0

Make sure we arent reinjecting more water than we are extracting

In [ ]:
diffobs = obs.loc[(obs.oname=="inc") & (obs.usecol=="totwel") & (obs.totim=="3651"),:]
assert diffobs.shape[0] == 1
obs.loc[diffobs.obsnme,"weight"] = 1.0
obs.loc[diffobs.obsnme,"obsval"] = 0.0
obs.loc[diffobs.obsnme,"obgnme"] = "less_than"

For an objective function, we want to minimize the total flux of water being used for both dewatering and reinjection.  That value comes from the modflow list file budget and the absolute sum of both wel packages (we added that bit way back in the pstfrom notebook)

In [ ]:
aobs = obs.loc[(obs.oname=="inc") & (obs.usecol=="abstotwel") & (obs.totim=="3651"),:]
assert aobs.shape[0] == 1
objname = aobs.obsnme.values[0]
#obs.loc[objname,"weight"] = 1.0
#obs.loc[objname,"obgnme"] = "less_than"
pst.pestpp_options["opt_obj_func"] = objname

In [ ]:
pst.control_data.noptmax = 3

In [ ]:
pst.write(os.path.join(t_d,"pest.pst"),version=2)

In [ ]:
num_workers=30
m_d = "master_opt0"

In [ ]:
pyemu.os_utils.start_workers(t_d, # the folder which contains the "template" PEST dataset
                            'pestpp-opt', #the PEST software version we want to run
                            'pest.pst', # the control file to use with PEST
                            num_workers=num_workers, #how many agents to deploy
                            worker_root='.', #where to deploy the agent directories; relative to where python is running
                            master_dir=m_d, #the manager directory
                            )

Ruh roh - our problem is infeasible!  that means its not possible to make the model meet both the strict pit gw level requirement and the post-closure GDE flux requirement #sad

So lets loosen things up a lil - what if we only need to be within 20% of the pre-development GDE flux - it is after all an imprecise measured value...

In [ ]:
predev_gde_val = obs.loc[forecasts[0],"obsval"] = obs.loc[forecasts[0].replace("time:40151","time:1"),"obsval"]
print(predev_gde_val)
rhs_val = predev_gde_val - ((predev_gde_val * 0.2))
print(rhs_val)
obs.loc[forecasts[0],"obsval"] = rhs_val

In [ ]:
pst.write(os.path.join(t_d,"pest.pst"),version=2)

In [ ]:
pyemu.os_utils.start_workers(t_d, # the folder which contains the "template" PEST dataset
                            'pestpp-opt', #the PEST software version we want to run
                            'pest.pst', # the control file to use with PEST
                            num_workers=num_workers, #how many agents to deploy
                            worker_root='.', #where to deploy the agent directories; relative to where python is running
                            master_dir=m_d, #the manager directory
                            )

How many "scenarios" would you have to run to do this same analysis #thinkaboutit